In [1]:
!git clone https://github.com/ViralLab/TurkishBERTweet.git

Cloning into 'TurkishBERTweet'...
remote: Enumerating objects: 1259, done.
remote: Counting objects: 100% (759/759), done.
remote: Compressing objects: 100% (469/469), done.
remote: Total 1259 (delta 423), reused 621 (delta 290), pack-reused 500 (from 1)
Receiving objects: 100% (1259/1259), 5.06 MiB | 31.00 MiB/s, done.
Resolving deltas: 100% (648/648), done.


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install peft 
!pip install transformers
!pip install urlextract

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 9.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [3]:
import sys
sys.path.append('./TurkishBERTweet')

In [4]:
from Preprocessor import preprocess

text = """Lab'ımıza "viral" adını verdik çünkü amacımız disiplinler arası sınırları aşmak ve aralarında yeni bağlantılar kurmak! 🔬 #ViralLab
https://varollab.com/"""

preprocessed_text = preprocess(text)
print(preprocessed_text)

lab'ımıza "viral" adını verdik çünkü amacımız disiplinler arası sınırları aşmak ve aralarında yeni bağlantılar kurmak! <emoji> mikroskop </emoji> <hashtag> virallab </hashtag> <http> varollab.com </http>


In [5]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from Preprocessor import preprocess

tokenizer = AutoTokenizer.from_pretrained("VRLLab/TurkishBERTweet")
turkishBERTweet = AutoModelForSequenceClassification.from_pretrained("VRLLab/TurkishBERTweet", num_labels=10)

text = """Lab'ımıza "viral" adını verdik çünkü amacımız disiplinler arası sınırları aşmak ve aralarında yeni bağlantılar kurmak! 💥🔬 #ViralLab #DisiplinlerArası #YenilikçiBağlantılar"""

preprocessed_text = preprocess(text)
input_ids = torch.tensor([tokenizer.encode(preprocessed_text)])

with torch.no_grad():
    features = turkishBERTweet(input_ids)  # Models outputs are now tuples

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/652M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at VRLLab/TurkishBERTweet and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Data Preprocessing and Cleaning

In [6]:
!pip install nltk
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint
#@title Turkish StopWords

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
train_classification_df = pd.read_csv("/kaggle/input/cs412-dataset/train-classification.csv")
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [8]:
train_data_path = "/kaggle/input/cs412-dataset/training-dataset.jsonl"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"] # Everything is under here except for posts
    username = profile["username"] #Get the username
    if username in username2_category: #We already know the category for this user.
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile #Add the profile data into this dictionary


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile


In [9]:
# Profile Dataframe
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,deparmedya,3170700063,Depar Medya,#mediaplanning #mediabuying #sosyalmedya,Local business,None,1167,192,True,False,...,None,None,LOCAL,False,False,https://instagram.fsaw2-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,kafesfirin,266439571,KAFES FIRIN,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,Brand,None,11997,17,True,False,...,None,None,BRAND,False,False,https://instagram.fada1-13.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [10]:
test_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [11]:
print("Profile DataFrame Columns:", train_profile_df.columns.tolist())
print("Count is:", len(train_profile_df.columns))


Profile DataFrame Columns: ['username', 'id', 'full_name', 'biography', 'category_name', 'post_count', 'follower_count', 'following_count', 'is_business_account', 'is_private', 'is_verified', 'highlight_reel_count', 'bio_links', 'entities', 'ai_agent_type', 'fb_profile_biolink', 'restricted_by_viewer', 'country_block', 'eimu_id', 'external_url', 'fbid', 'has_clips', 'hide_like_and_view_counts', 'is_professional_account', 'is_supervision_enabled', 'is_guardian_of_viewer', 'is_supervised_by_viewer', 'is_supervised_user', 'is_embeds_disabled', 'is_joined_recently', 'business_address_json', 'business_contact_method', 'business_email', 'business_phone_number', 'business_category_name', 'overall_category_name', 'category_enum', 'is_verified_by_mv4b', 'is_regulated_c18', 'profile_pic_url', 'should_show_category', 'should_show_public_contacts', 'show_account_transparency_details', 'profile_picture_base64']
Count is: 44


In [12]:
username2posts_train["deparmedya"][0]

{'caption': 'Cumhuriyetimizin 100.yılı kutlu olsun♾️🇹🇷',
 'comments_count': 0,
 'id': '17990918969458720',
 'like_count': 6,
 'media_type': 'IMAGE',
 'media_url': 'https://scontent-sof1-2.cdninstagram.com/v/t51.29350-15/396342908_267936919574308_4264417069827989599_n.jpg?_nc_cat=107&ccb=1-7&_nc_sid=c4dd86&_nc_ohc=IynXuQSoOT8AX9RSy20&_nc_ht=scontent-sof1-2.cdninstagram.com&edm=AL-3X8kEAAAA&oh=00_AfA8OKAM0MY9tqg6dw8C8I5TJp4SHPBp-VlNXrFAh2agqg&oe=6563581C',
 'timestamp': '2023-10-29 09:12:30'}

In [13]:
list(username2posts_train["deparmedya"][0].keys())

['caption',
 'comments_count',
 'id',
 'like_count',
 'media_type',
 'media_url',
 'timestamp']

In [14]:
username = "deparmedya"
user_profile = username2profile_train.get(username, {})
biography = user_profile.get("biography", "")
print(preprocess(biography))

<hashtag> mediaplanning </hashtag> <hashtag> mediabuying </hashtag> <hashtag> sosyalmedya </hashtag>


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji
from scipy.sparse import hstack


# to keep the label order
train_usernames = []
train_caption_bio_combined = []
train_captions = []
train_bio = []

max_total_tokens_for_user_captions = 0 # To keep the total number of tokens, if > 128, then we need to do something.
for username, posts in username2posts_train.items():
  train_usernames.append(username)
  user_profile = username2profile_train.get(username, {})
  user_biography = user_profile.get("biography") or ""
  user_biography = preprocess(user_biography) #using the preprocessor of the model
  train_bio.append(user_biography)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue
    post_caption = preprocess(post_caption)
    if post_caption != "":
      cleaned_captions.append(post_caption)
  user_post_captions = " ".join(cleaned_captions)
  if not user_post_captions.strip():
    print("User " + username + " from training has empty captions.")

  train_captions.append(user_post_captions)

y_train = [username2_category.get(uname, "NA").lower() for uname in train_usernames]


User birguzeladam from training has empty captions.
User touchdownistanbul from training has empty captions.
User mks_kilit_sistemleri from training has empty captions.
User belediyesikose from training has empty captions.
User esqtekstil from training has empty captions.


In [16]:
len(y_train)

2741

In [17]:
train_captions[0]

"cumhuriyetimizin 100.yılı kutlu olsun <emoji> sonsuzluk </emoji> <emoji> bayrak_tunus </emoji> oriflame duologi lansmanı <hashtag> isveçtengelengüzellik </hashtag> <hashtag> oriflameilesaçbakımdevrimi </hashtag> <hashtag> oriflameilesaçbakımdevrimi </hashtag> <emoji> sıkıştıran_el_koyu_ten_tonu </emoji> <emoji> sıkıştıran_el_koyu_ten_tonu </emoji> <hashtag> oriflameilesaçbakımdevrimi </hashtag> 07agustos’23 oriflameturkiye 07 agustos’23 <hashtag> oriflameturkiye </hashtag> <hashtag> duoloji </hashtag> oriflame <hashtag> duoloji </hashtag> muhteşem saçların sırrı <hashtag> duoloji </hashtag> oriflameturkiye goe elektirikli motorsiklet ile , sürdürülebilir bir yaşamın elçisi olun! <hashtag> goeilegeleceğeyönver </hashtag> <hashtag> goeilesürdürülebilirgelecek </hashtag> <hashtag> easytorideeasytolove </hashtag> goe_mobility <emoji> motosiklet </emoji> 7 haziran’23 <hashtag> yazamerhaba </hashtag> feridaistanbul 7 haziran’23 <hashtag> yazamerhaba </hashtag> feridaistanbul <hashtag> yazam

In [18]:
train_bio[0]

'<hashtag> mediaplanning </hashtag> <hashtag> mediabuying </hashtag> <hashtag> sosyalmedya </hashtag>'

In [19]:
train_usernames[0]

'deparmedya'

In [20]:
categories = [
    'mom and children',
    'food',
    'travel',
    'gaming',
    'fashion',
    'health and lifestyle',
    'tech',
    'entertainment',
    'sports',
    'art'
]

# Create label2id and id2label
label2id = {label: idx for idx, label in enumerate(categories)}
id2label = {idx: label for idx, label in enumerate(categories)}

print("label2id:", label2id)
print("id2label:", id2label)


label2id: {'mom and children': 0, 'food': 1, 'travel': 2, 'gaming': 3, 'fashion': 4, 'health and lifestyle': 5, 'tech': 6, 'entertainment': 7, 'sports': 8, 'art': 9}
id2label: {0: 'mom and children', 1: 'food', 2: 'travel', 3: 'gaming', 4: 'fashion', 5: 'health and lifestyle', 6: 'tech', 7: 'entertainment', 8: 'sports', 9: 'art'}


In [21]:
len(label2id)

10

In [22]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset

from peft import (
    PeftModel,
    PeftConfig,
)

texts = []
for bio, caption in zip(train_bio, train_captions):
    combined_text = bio.strip() + " " + caption.strip()
    texts.append(combined_text)

In [23]:
numeric_labels = [label2id[label_str] for label_str in y_train]
len(numeric_labels)

2741

In [24]:
texts[0]

"<hashtag> mediaplanning </hashtag> <hashtag> mediabuying </hashtag> <hashtag> sosyalmedya </hashtag> cumhuriyetimizin 100.yılı kutlu olsun <emoji> sonsuzluk </emoji> <emoji> bayrak_tunus </emoji> oriflame duologi lansmanı <hashtag> isveçtengelengüzellik </hashtag> <hashtag> oriflameilesaçbakımdevrimi </hashtag> <hashtag> oriflameilesaçbakımdevrimi </hashtag> <emoji> sıkıştıran_el_koyu_ten_tonu </emoji> <emoji> sıkıştıran_el_koyu_ten_tonu </emoji> <hashtag> oriflameilesaçbakımdevrimi </hashtag> 07agustos’23 oriflameturkiye 07 agustos’23 <hashtag> oriflameturkiye </hashtag> <hashtag> duoloji </hashtag> oriflame <hashtag> duoloji </hashtag> muhteşem saçların sırrı <hashtag> duoloji </hashtag> oriflameturkiye goe elektirikli motorsiklet ile , sürdürülebilir bir yaşamın elçisi olun! <hashtag> goeilegeleceğeyönver </hashtag> <hashtag> goeilesürdürülebilirgelecek </hashtag> <hashtag> easytorideeasytolove </hashtag> goe_mobility <emoji> motosiklet </emoji> 7 haziran’23 <hashtag> yazamerhaba <

In [25]:
model_name = "dbmdz/bert-base-turkish-uncased"
# model_name = "dbmdz/bert-base-turkish-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=10,            # <-- your 10 labels
    id2label=id2label,
    label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from sklearn.model_selection import train_test_split
# Apply an 80-20 split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, numeric_labels, test_size=0.2, random_state=42
)

# Output the results
print("Train texts:", len(train_texts))
print("Test texts:", len(test_texts))
print("Train labels:", len(train_labels))
print("Test labels:", len(test_labels))

Train texts: 2192
Test texts: 549
Train labels: 2192
Test labels: 549


In [27]:
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset  = Dataset.from_dict({"text": test_texts, "label": test_labels})

In [28]:
def chunk_text(examples, chunk_size=512):
    """
    Splits each example's text into chunks of up to `chunk_size` words,
    and replicates the label for each chunk.
    """
    new_texts = []
    new_labels = []

    # examples["text"] is now a *list* of strings
    # examples["label"] is now a *list* of labels
    for text, label in zip(examples["text"], examples["label"]):
        words = text.split()
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i : i + chunk_size])
            new_texts.append(chunk)
            new_labels.append(label)

    return {"text": new_texts, "label": new_labels}

In [29]:
# Apply chunking to train and test sets
train_dataset = train_dataset.map(chunk_text, batched=True)
test_dataset  = test_dataset.map(chunk_text,  batched=True)

Map:   0%|          | 0/2192 [00:00<?, ? examples/s]

Map:   0%|          | 0/549 [00:00<?, ? examples/s]

In [30]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",  # or "longest", or dynamic padding
        max_length=512         # or 256, or 512 depending on your data
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


# The Trainer expects the columns "input_ids", "attention_mask", and "labels"
train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

# set the format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/8420 [00:00<?, ? examples/s]

Map:   0%|          | 0/2101 [00:00<?, ? examples/s]

In [31]:
!pip install evaluate
import numpy as np
from evaluate import load
metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [33]:
from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir="best_performing_TurkishBERTweet",
    eval_strategy="steps",    # Evaluate at regular steps
    eval_steps=50,                  # Evaluation frequency
    save_strategy="steps",
    save_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    logging_steps=10,               # Log every 10 steps
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    disable_tqdm=False,             # Enable progress bars
    report_to=["none"]              # Disable integration reports
)
# Initialize the EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,      # Stop after 2 evaluations with no improvement
    early_stopping_threshold=0.0    # Minimum change to qualify as improvement
)

# Ensure all model parameters are contiguous
for param in model.parameters():
    param.data = param.data.contiguous()

from torch.optim import AdamW

# Define weight decay parameters
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,  # Adjust weight decay as needed
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

# Initialize the optimizer with the custom learning rate
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

from transformers import get_linear_schedule_with_warmup

# Calculate the total number of training steps
num_epochs = training_args.num_train_epochs
num_training_steps = len(train_dataset) // training_args.per_device_train_batch_size * num_epochs

# Define the number of warmup steps
num_warmup_steps = int(0.1 * num_training_steps)  # 10% of training steps for warmup

# Initialize the scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)


In [34]:
print(model)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,       # Include validation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),  # Pass the optimizer and scheduler here
    callbacks=[early_stopping]      # Add the early stopping callback
)

trainer.train()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Step,Training Loss,Validation Loss,Accuracy
50,2.385200,2.380813,0.063779
100,2.338500,2.308991,0.080438
150,2.198100,2.208130,0.194193
200,2.065000,2.145902,0.201333
250,2.091600,2.092469,0.265112
300,2.069800,2.067296,0.273203
350,2.043400,2.039759,0.286054
400,2.042600,1.985015,0.322703
450,1.894900,1.889013,0.365540
500,1.796700,1.723411,0.468348


TrainOutput(global_step=1650, training_loss=1.352440616723263, metrics={'train_runtime': 1860.1332, 'train_samples_per_second': 45.266, 'train_steps_per_second': 5.661, 'total_flos': 3472262876995584.0, 'train_loss': 1.352440616723263, 'epoch': 1.566951566951567})

In [35]:
predictions = trainer.predict(test_dataset)
print(predictions.metrics)


{'test_loss': 0.9354445338249207, 'test_accuracy': 0.7172774869109948, 'test_runtime': 31.542, 'test_samples_per_second': 66.61, 'test_steps_per_second': 8.338}


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji
from scipy.sparse import hstack


# to keep the label order
test_usernames = []
test_captions = []
test_bio = []

for username, posts in username2posts_test.items():
  test_usernames.append(username)
  user_profile = username2profile_test.get(username, {})
  user_biography = user_profile.get("biography") or ""
  user_biography = preprocess(user_biography) #using the preprocessor of the model
  test_bio.append(user_biography)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue
    post_caption = preprocess(post_caption)
    if post_caption != "":
      cleaned_captions.append(post_caption)
  user_post_captions = " ".join(cleaned_captions)
  if not user_post_captions.strip():
    print("User " + username + " from test has empty captions.")

  test_captions.append(user_post_captions)



User sanatdan_ from test has empty captions.
User modamizbir from test has empty captions.
User deeprockbar35 from test has empty captions.
User flamingo_fashionn from test has empty captions.
User dogukandrl from test has empty captions.


In [37]:
test_texts = []
for bio_text, captions_text in zip(test_bio, test_captions):
    combined_text = bio_text.strip() + " " + captions_text.strip()
    test_texts.append(combined_text)

In [38]:
test_texts[0]

'beyaz yakalıların dünyasına hoşgeldiniz <emoji> sırıtan_yüz </emoji> <emoji> sırıtan_yüz </emoji> <emoji> sırıtan_yüz </emoji> bu diyaloğun yaşanmadığı bir online toplantı olmaz olamaz <emoji> sevinç_gözyaşlarıyla_yüzleşmek </emoji> evet ocak ayında beyaz yakalı whatsup gruplarında en çok sorulan soru, <emoji> sırıtan_yüz </emoji> hayır post gecikmeli değil, hala öğrenememiş olan binlerce kişi olduğunu söyleyebilirim ama ispat edemem <emoji> kötülük_görmeyen_maymun </emoji> <emoji> sırıtan_yüz </emoji> yine yuzlercesini gorecegimiz maillerden biri <emoji> sırıtan_yüz </emoji> <emoji> kötülük_görmeyen_maymun </emoji> i̇yi haftalar ! <emoji> sırıtan_yüz </emoji> bir iç ses <emoji> sırıtan_yüz </emoji> 10 dk kahve içmek mi <emoji> sırıtan_yüz </emoji> <emoji> kötülük_görmeyen_maymun </emoji> geldi hesap kitap ayları <emoji> sırıtan_yüz </emoji> <emoji> kötülük_görmeyen_maymun </emoji> yaşayan bilir. sen, ben, hepimiz <emoji> kötülük_görmeyen_maymun </emoji> . . <hashtag> beyazyakalı </ha

In [40]:
test_data_path = "/kaggle/input/cs412-dataset/test-classification-round3.dat"

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

In [57]:
test_unames[:10]

['livapastanesi',
 'barisgross',
 'tusasshop',
 'etolyadigital',
 'tugrulonur',
 'tulugozlu',
 'gokidy',
 'cengizgumus_official',
 'krossbisiklet',
 'haribochamallows']

In [59]:
test_data_text = []
for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    test_data_text.append(test_texts[index])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      test_data_text.append(texts[index])
    except Exception as e:
      print(uname)




In [65]:
len(test_data_text)
len(test_unames)

1000

In [107]:
from collections import defaultdict
import torch

# Step 1: Create a Dataset for test data
real_test_dataset = Dataset.from_dict({"text": test_data_text, "username": test_unames})
def chunk_text_no_label(examples, chunk_size=512):
    new_texts = []
    new_usernames = []

    for text, uname in zip(examples["text"], examples["username"]):
        words = text.split()
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i : i + chunk_size])
            new_texts.append(chunk)
            new_usernames.append(uname)

    return {"text": new_texts, "username": new_usernames}
    
real_test_dataset = real_test_dataset.map(
    chunk_text_no_label, 
    batched=True, 
    remove_columns=real_test_dataset.column_names
)

def tokenize_function(examples):
    encodings = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    # Just return whatever the tokenizer gives plus the username
    encodings["username"] = examples["username"]  
    return encodings

real_test_dataset = real_test_dataset.map(tokenize_function, batched=True)

# Store everything in a plain arrow/pandas structure
df_test = real_test_dataset.to_pandas()

real_test_dataset.set_format(
    type="torch", 
    columns=["input_ids", "attention_mask"]
)




# Step 5: Run predictions
predictions = trainer.predict(real_test_dataset)

import numpy as np
import torch

logits = predictions.predictions  # shape [n_chunks, n_classes]
logits_df = pd.DataFrame(logits, columns=list(range(logits.shape[1])))
df_test = real_test_dataset.to_pandas()
logits_df["username"] = df_test["username"].values
# 3) Average per username
avg_logits = logits_df.groupby("username").mean()  # shape: [n_users, n_classes]

# 4) Identify highest logit column => idxmax
pred_class_per_user = avg_logits.idxmax(axis=1)  # this will give an integer column name
user2pred_class = pred_class_per_user.to_dict()

'''pred_ids = np.argmax(logits, axis=-1)  # shape [n_chunks]
df_test["pred_id"] = pred_ids
user2chunks = df_test.groupby("username")["pred_id"].agg(lambda x: x.value_counts().index[0])
# Convert logits to a DataFrame
logits_df = pd.DataFrame(logits)
logits_df["username"] = df_test["username"].values

# Now group by user and average the logits
avg_logits = logits_df.groupby("username").mean()

# For each user, pick the class with the highest average logit
pred_class_per_user = avg_logits.idxmax(axis=1)


user2pred_class = user2chunks.to_dict()
'''



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4274 [00:00<?, ? examples/s]

'pred_ids = np.argmax(logits, axis=-1)  # shape [n_chunks]\ndf_test["pred_id"] = pred_ids\nuser2chunks = df_test.groupby("username")["pred_id"].agg(lambda x: x.value_counts().index[0])\n# Convert logits to a DataFrame\nlogits_df = pd.DataFrame(logits)\nlogits_df["username"] = df_test["username"].values\n\n# Now group by user and average the logits\navg_logits = logits_df.groupby("username").mean()\n\n# For each user, pick the class with the highest average logit\npred_class_per_user = avg_logits.idxmax(axis=1)\n\n\nuser2pred_class = user2chunks.to_dict()\n'

In [104]:
id2label

{0: 'mom and children',
 1: 'food',
 2: 'travel',
 3: 'gaming',
 4: 'fashion',
 5: 'health and lifestyle',
 6: 'tech',
 7: 'entertainment',
 8: 'sports',
 9: 'art'}

In [109]:
len(user2pred_class)

1000

In [110]:
import json
mapped_dict = {key: id2label[value] for key, value in user2pred_class.items()}


In [111]:
bert_results_df = pd.DataFrame(list(mapped_dict.items()), columns=['username', 'predicted_class'])
bert_results_df.head()

,username,predicted_class
0,01burdaavm,food
1,1001sanat,art
2,1983beyoglu,food
3,253binyasin,entertainment
4,35likmeyhaneizmir,food


In [112]:
count = 0
for username in username2posts_train:
  if username in test_unames:
    count += 1
print(f"There are {count} overlapping instances in the training dataset and 1000 classification test instances")

There are 266 overlapping instances in the training dataset and 1000 classification test instances


In [113]:
# Iterate over rows in the dataframe
overlapping_count = 0
correct_prediction = 0
wrong_prediction = 0
for index, row in bert_results_df.iterrows():
    username = row['username']
    predicted_class = row['predicted_class']
    if username in username2_category:
        actual_class = username2_category.get(username)
        overlapping_count += 1
        if predicted_class == actual_class:
            print(f"The user {username} is in the training data, and the prediction {predicted_class} is true")
            correct_prediction += 1
        else:
            print(f"The user {username} is in the training data, and the prediction {predicted_class} is NOT true. True label is {actual_class}")
            bert_results_df.at[index, 'predicted_class'] = actual_class
            wrong_prediction += 1

print(f"There are {overlapping_count} overlapping instances in the training dataset and 1000 classification test instances\n")
print(f"{correct_prediction} are correctly classified, {wrong_prediction} are incorrectly classified.")



The user 1001sanat is in the training data, and the prediction art is NOT true. True label is entertainment
The user _helinkandemir is in the training data, and the prediction entertainment is NOT true. True label is health and lifestyle
The user acarflowers is in the training data, and the prediction art is true
The user adnanturankb is in the training data, and the prediction health and lifestyle is true
The user agoraavm_antalya is in the training data, and the prediction entertainment is true
The user ahmetmaranki is in the training data, and the prediction food is NOT true. True label is health and lifestyle
The user akcoat_official is in the training data, and the prediction tech is true
The user akhisarpresshaber is in the training data, and the prediction health and lifestyle is NOT true. True label is entertainment
The user alfaglb is in the training data, and the prediction food is true
The user alialtuntasbaskan is in the training data, and the prediction health and lifestyl

In [ ]:
mapped_df = pd.DataFrame(list(mapped_dict.items()), columns=["Key", "Category"])

# Create a dictionary from the DataFrame
mapped_df = mapped_df.set_index('Key')['Category'].to_dict()
# Get distinct values from the dictionary
distinct_values = list(set(mapped_df.values()))
distinct_values

In [ ]:
# Iterate over rows in the dataframe
overlapping_count = 0
correct_prediction = 0
wrong_prediction = 0
for index, row in mapped_df.iterrows():
    username = row['Key']
    predicted_class = row['Value']
    if username in username2_category:
        actual_class = username2_category.get(username)
        overlapping_count += 1
        if predicted_class == actual_class:
            print(f"The user {username} is in the training data, and the prediction {predicted_class} is true")
            correct_prediction += 1
        else:
            print(f"The user {username} is in the training data, and the prediction {predicted_class} is NOT true. True label is {actual_class}")
            predictions_df.at[index, 'predicted_class'] = actual_class
            wrong_prediction += 1

print(f"There are {overlapping_count} overlapping instances in the training dataset and 1000 classification test instances\n")
print(f"{correct_prediction} are correctly classified, {wrong_prediction} are incorrectly classified.")



In [80]:
# Load the dataset
train_classification_df = pd.read_csv("/kaggle/input/cs412-dataset/train-classification.csv")
# Rename the columns
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Get the list of distinct category labels
distinct_categories = train_classification_df['category'].unique().tolist()
distinct_categories
distinct_categories.remove("Health and lifestyle")
distinct_categories


['Mom and Children',
 'Food',
 'Travel',
 'Gaming',
 'Fashion',
 'Health and Lifestyle',
 'Tech',
 'Entertainment',
 'Sports',
 'Art']

In [85]:
# Mapping values from distinct_values to distinct_categories
mapping = {}
for value in distinct_values:
    for category in distinct_categories:
        # Case-insensitive match
        if value.lower() == category.lower():
            mapping[value] = category
            break

In [86]:
mapping

{'health and lifestyle': 'Health and Lifestyle',
 'tech': 'Tech',
 'entertainment': 'Entertainment',
 'travel': 'Travel',
 'sports': 'Sports',
 'art': 'Art',
 'food': 'Food',
 'mom and children': 'Mom and Children',
 'fashion': 'Fashion'}

In [87]:
updated_mapped_df = {
    key: mapping.get(value, value) for key, value in mapped_df.items()
}

In [92]:
len(updated_mapped_df)

1000

In [ ]:
# Define the output file path
output_file = "prediction-classification-round3.json"

# Write the dictionary to a JSON file
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(updated_predictions_dict, f, indent=4, ensure_ascii=False)

print(f"Predictions have been written to {output_file}")

In [ ]:
# Step 6: Aggregate predictions by username
username_predictions = defaultdict(list)
for example, predicted_class in zip(test_dataset, predicted_classes):
    username_predictions[example["username"]].append(predicted_class)

# Step 7: Assign final prediction per username (e.g., majority voting)
final_predictions = {}
for username, preds in username_predictions.items():
    # Use majority vote for final prediction
    final_predictions[username] = max(set(preds), key=preds.count)

# final_predictions now contains the mapping of username -> predicted class